# Quantum Convolutional Networks
- [x] **torch dataset**
- [x] process bangla numbers data 
- [x] **quanvolution** circuit testing
- [x] create a hybrid model: quantum-classical
- [ ] test results

# Prepare the dataset to work with torch and qiskit

In [ ]:
#------------------------------
# imports
#------------------------------
import os
import pandas as pd
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset,DataLoader
from torchvision.transforms import ToTensor
#------------------------------
# setting up seed and checking GPU
#------------------------------
'''
    as we will be using a random_circuit this seed is needed
'''
seed = 47
np.random.seed(seed)        
torch.manual_seed(seed)     

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)
    print('cuda index:', torch.cuda.current_device())
    print('GPU:', torch.cuda.get_device_name())
else:
    DEVICE = torch.device('cpu')
    print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)



In [ ]:
#--------------------------
# parameters
#--------------------------
BATCH_SIZE    = 10
EPOCHS        = 10     
PREPROCESS    = True           # If False, skip quantum processing

n_train       = 50     # Size of the train dataset
n_test        = 30     # Size of the test dataset
n_class       = 10     

In [ ]:
#------------------------------
# dataset class for torch
#------------------------------
class BanglaNumbersDataset(Dataset):
    def __init__(self, 
                 img_dir,
                 df,
                 data_dim=28,
                 transform=None):
        # for labels and path
        self.df        = df
        # directory
        self.img_dir   = img_dir
        # transformations
        self.transform = transform
        # dimension
        self.data_dim  = data_dim

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # define path
        img_path = os.path.join(self.img_dir, f"{self.df.iloc[idx, 0]}.bmp")
        # read
        image = cv2.imread(img_path,0)
        image = cv2.resize(image,(self.data_dim,self.data_dim))
        label = self.df.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
        return image,label

#--------------------------
# test train balanced split
#--------------------------

# read
df=pd.read_csv("/kaggle/input/banglaq/numbers.csv")
bangla_labels=sorted(list(df.label.unique()))
# normalize labels
df.label=df.label.apply(lambda x: bangla_labels.index(x))

train_dfs=[]
test_dfs =[]

'''
    Note:since there are thousands of data we wont bother with df empty checks or head tail duplications
'''
# collect class-wise balanced data
for label in df.label.unique():
    _df=df.loc[df.label==label]
    train_dfs.append(_df.head(int(n_train/n_class)))
    test_dfs.append(_df.tail(int(n_test/n_class)))
# concat and shuffle
train_df=pd.concat(train_dfs,ignore_index=True)
train_df=train_df.sample(frac=1)

test_df =pd.concat(test_dfs,ignore_index=True)
test_df=test_df.sample(frac=1)

print(f" Total Data:{len(df)} \n Train Data:{len(train_df)} \n Test Data:{len(test_df)}")
#----------------------------
# datasets and dataloaders
#------------------------------
train_ds        =      BanglaNumbersDataset(img_dir="/kaggle/input/banglaq/numbers/",
                                            df=train_df,
                                            transform=ToTensor())
test_ds         =      BanglaNumbersDataset(img_dir="/kaggle/input/banglaq/numbers/",
                                            df=test_df,
                                            transform=ToTensor())


train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)
#----------------------------
# display dataset
#------------------------------                                            
# Display image and label.
train_images,train_labels= next(iter(train_dataloader))
# train_images=np.array(batch["image"]) 
# train_labels=np.array(batch["label"]) 
print(f"Images batch shape: {train_images.shape}")
print(f"Labels batch shape: {train_labels.shape}")
img   = train_images[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")
print("unique values:",np.unique(img))

# Libraries and setup

In [ ]:
from IPython.display import clear_output
!pip install qiskit==0.24.0
!pip install qiskit-aer-gpu
!pip install -U numpy
!pip install pylatexenc
clear_output()

# Quanvolutional Networks
* **Step-1**:Take a small region of the input image and embed in into a quantum circuit 
```python
This can be done with parameterized rotations applied to the qubits initialized in the ground state. i.e-
theta           = list of circuit parameter theta for all qubits 
number of qubits= kernel_size square 
for each qubit: 
    apply Single-qubit rotation about the X axis.
```
* **Reminder** :

\begin{align}\begin{aligned}\newcommand{\th}{\frac{\theta}{2}}\\\begin{split}RX(\theta) = exp(-i \th X) =
    \begin{pmatrix}
        \cos{\th}   & -i\sin{\th} \\
        -i\sin{\th} & \cos{\th}
    \end{pmatrix}\end{split}\end{aligned}\end{align}
  

* **Step-2**: Perform a quantum computaion associated with a unitary. 
> Simply using VQC (variational quantum circuit) or a Random-circuit is enough

* **Step-3**:Measure the system to get classical expectation values. 


In [ ]:
import qiskit
from qiskit import IBMQ
from qiskit.visualization import *
from qiskit.circuit.random import random_circuit

# formulate and check the circuit
* https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html
* https://qiskit.org/documentation/stubs/qiskit.circuit.random.random_circuit.html

In [ ]:
from qiskit.circuit import Parameter
from qiskit.compiler import transpile
#----------------------
# circuit definitions
#----------------------
kernel_size=2
# needed number of qbits
n_qubits = kernel_size** 2
# initialize circuit
_circuit = qiskit.QuantumCircuit(n_qubits)
# get rotation theta at ground state
params = [Parameter('θ_{}'.format(i+1)) for i in range(n_qubits)]
# apply rotation
for i in range(n_qubits):
    # theta and qubit
    _circuit.rx(params[i],i)
# add barrier
_circuit.barrier()
# random circuit
_circuit += random_circuit(n_qubits, 2)
# add measurements
_circuit.measure_all()
print(_circuit.parameters)
_circuit.draw(output='mpl')


In [ ]:
#-------------------------
# running the circuit
#------------------------
shots      = 100
threshold  = 127
# create backend
#backend = qiskit.Aer.get_backend('qasm_simulator')
backend = qiskit.providers.aer.QasmSimulator(method = "statevector_gpu")
# create random data 2x2 data
data = torch.tensor([[0, 200], [100, 255]])
# reshape
data = torch.reshape(data, (1,n_qubits))

# encoding data to parameters
## calculate theta
'''
    val > self.threshold  : |1> - rx(pi)
    val <= self.threshold : |0> - rx(0)
'''
                
thetas = []
for dat in data:
    theta = []
    for val in dat:
        if val > threshold:
            theta.append(np.pi)
        else:
            theta.append(0)
    thetas.append(theta)
# bind the parameters
param_dict = dict()
for theta in thetas:
    for i in range(n_qubits):
        param_dict[params[i]] = theta[i]
param_binds = [param_dict]
print(_circuit.parameters)
print(param_binds)
# # transpile the circuit
_circuit=transpile(_circuit)
# execute quantum circuit
'''
    since we are using simulator we dont have to keep track of the job
'''
result = qiskit.execute(_circuit, 
                         backend, 
                         shots = shots , 
                         parameter_binds = param_binds).result().get_counts()
# decoding the result
counts = 0
for key, val in result.items():
    cnt = sum([int(char) for char in key])
    counts += cnt * val

# Compute probabilities for each state
probabilities = counts / (shots  * n_qubits)
print("Convolution result:",probabilities)

# Create A wrapper class for torch to use

In [ ]:
class QCircuit:
    def __init__(self, 
                 kernel_size, 
                 backend, 
                 shots, 
                 threshold):
        '''
            the quantum circuit class to execute the convolution operation
            args:
                kernel_size   =   size of (same as classical conv) kernel/ the image dimension patch
                backend       =   the quantum computer hardware to use (only simulator is used here)
                shots         =   how many times to run the circuit to get a probability distribution
                threshold     =   the threshold value (0-255) to assign theta
        '''
        # the number of qubits needed
        self.n_qubits = kernel_size ** 2
        # initiate the circuit
        self._circuit = qiskit.QuantumCircuit(self.n_qubits)
        # parameters
        self._params = [Parameter('θ_{}'.format(i)) for i in range(self.n_qubits)]

        for i in range(self.n_qubits):
            self._circuit.rx(self._params[i], i)
        
        self._circuit.barrier()
        # add unitary random circuit
        self._circuit += random_circuit(self.n_qubits, 2)
        self._circuit.measure_all()
        # initialize
        self.backend   = backend
        self.shots     = shots
        self.threshold = threshold

    def run(self, data):
        # reshape input data-> [1, kernel_size, kernel_size] -> [1, self.n_qubits]
        data = torch.reshape(data, (1, self.n_qubits))
        # encoding data to parameters
        thetas = []
        for dat in data:
            theta = []
            for val in dat:
                if val > self.threshold:
                    theta.append(np.pi)
                else:
                    theta.append(0)
            thetas.append(theta)
        # for binding parameters
        param_dict = dict()
        for theta in thetas:
            for i in range(self.n_qubits):
                param_dict[self._params[i]] = theta[i]
        param_binds = [param_dict]

        # execute random quantum circuit
        result = qiskit.execute(self._circuit, 
                                self.backend, 
                                shots = self.shots, 
                                parameter_binds = param_binds).result().get_counts()
            
        # decoding the result
        counts = 0
        for key, val in result.items():
            cnt = sum([int(char) for char in key])
            counts += cnt * val

        # Compute probabilities for each state
        probabilities = counts / (self.shots * self.n_qubits)
        
        return probabilities

# Torch forward and backward

In [ ]:
#----------------------------
# torch imports
#----------------------------

from torch.autograd import Function
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
#----------------------------
# layer adoptation
#----------------------------
'''
    see the textbook reference under materials
    the backward is used as it is- no change needed
'''
class QuanvFunction(Function):
    """ Quanv function definition """
    
    @staticmethod
    def forward(ctx, 
                inputs, 
                in_channels, 
                out_channels, 
                kernel_size, 
                quantum_circuits, 
                shift,
                verbose=True):
        """ Forward pass computation """
        # input  shape : (-1, 1, 28, 28)
        # otuput shape : (-1, 6, 24, 24)
        ctx.in_channels      = in_channels
        ctx.out_channels     = out_channels
        ctx.kernel_size      = kernel_size
        ctx.quantum_circuits = quantum_circuits
        ctx.shift            = shift

        # adjust length 
        _, _, len_x, len_y = inputs.size()
        len_x = len_x - kernel_size + 1
        len_y = len_y - kernel_size + 1
        
        # this calculates the conv results for nxn patches
        features = []
        ## loop over the images
        for input in inputs:
            feature = []
            ## loop over the circuits
            for circuit in quantum_circuits:
                # save the results
                xys = []
                for x in range(len_x):
                    ys = []
                    for y in range(len_y):
                        # get the patches
                        data = input[0, x:x+kernel_size, y:y+kernel_size]
                        # store the results
                        res=circuit.run(data)
                        ys.append(res)
                    xys.append(ys)
                feature.append(xys)
            features.append(feature)
        # construct the tensor
        result = torch.tensor(features)

        ctx.save_for_backward(inputs, result)
        return result
        
    @staticmethod
    def backward(ctx, grad_output): 
        """ Backward pass computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist())
        
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left  = ctx.quantum_circuit.run(shift_left[i])
            
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            gradients.append(gradient)
        gradients = np.array([gradients]).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None

#----------------------------
# module
#----------------------------
class Quanv(nn.Module):
    """ Quanvolution(Quantum convolution) layer definition """
    
    def __init__(self, 
                 in_channels, 
                 out_channels, 
                 kernel_size, 
                 backend=qiskit.providers.aer.QasmSimulator(method = "statevector_gpu"), 
                 shots=100, 
                 shift=np.pi/2):
        
        super(Quanv, self).__init__()
        
        self.quantum_circuits = [QCircuit(kernel_size=kernel_size, 
                                          backend=backend, 
                                          shots=shots, 
                                          threshold=0.5) for i in range(out_channels)]
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.kernel_size  = kernel_size
        self.shift        = shift
        
    def forward(self, inputs):
        return QuanvFunction.apply(inputs, 
                                   self.in_channels, 
                                   self.out_channels, 
                                   self.kernel_size,
                                   self.quantum_circuits, 
                                   self.shift)

# Model

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.quanv = Quanv(1, 6, kernel_size=5)
        self.conv = nn.Conv2d(6, 16, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.quanv(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

model = Net()
#summary(model,(1,28,28))
print(model)

# Train

In [ ]:

# params
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()
epochs = 20
loss_list = []

# train the model
model.train()

for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target) in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        # Calculating loss
        loss = loss_func(output, target)
        # Backward pass
        loss.backward()
        # Optimize the weights
        optimizer.step()
        total_loss.append(loss.item())
    loss_list.append(sum(total_loss)/len(total_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(100. * (epoch + 1) / epochs, loss_list[-1]))

# Evaluation Code 

In [ ]:

model.eval()
with torch.no_grad():
    correct = 0
    for batch_idx, (data, target) in enumerate(test_dataloader):
        data = data.cuda()
        target = target.cuda()
        output = model(data).cuda()
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = loss_func(output, target)
        total_loss.append(loss.item())
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_loader) * 100 / batch_size)
        )

# References and Materials
* [Previous Work(Basics to get things started)](https://github.com/mnansary/pyQIBM)
* [Paper: Quantum convolutional neural networks](https://www.nature.com/articles/s41567-019-0648-8)
* [Video Explaination of the paper](https://www.youtube.com/watch?v=C7kK5m7d10Q)
* [Tensorflow-quantum-github](https://github.com/tensorflow/quantum/tree/v0.4.0)
* [CNN with Tensorflow-quantum](https://www.tensorflow.org/quantum/tutorials/qcnn#1_build_a_qcnn)
* [Qiskit-pytorch hybrid basic (From Textbook)](https://qiskit.org/textbook/ch-machine-learning/machine-learning-qiskit-pytorch.html)
* [Paper:Quanvolutional Neural Networks: Powering Image Recognition with Quantum Circuits](https://arxiv.org/pdf/1904.04767.pdf)


Thanks to-
* [Dohun Kim](https://github.com/yh08037/)
* [pennylane.ai](pennylane.ai)


# How Does Tensoflow Work? (OpKernel,OpContext): Basic Reminder C++
* Defines a graph
* Call DirectSession::Run() Internally:
* The graph gets processed (split/optimized/etc...)
* Executors get created for each subgraph
* A RunState gets created
* Inputs get sent (/stored) in the Rendevouz
* Executor::RunAsync() for all executors
* FillContextMap assigns contexts to all nodes
* The Process(TaggedNode,..) function gets scheduled to the ThreadPool for all root nodes
There:
* The input tensors and parameters get prepared (and an OpKernel and OpContext get created)
* Device::Compute(kernel,context,..) gets called (sync or async)
* Device::Compute, when the device is a GPU device, has a different implementation: it checks if the inputs are in a context different from the on for this node (and wait for their completion via an event hook if they're not)
* Then the current stream gets changed (global for CUDA I think) 
* The OpKernel::Compute method gets called where if there's any kernel to be executed it is practically queued on the stream (which is the equivalent of a command queue in OpenCL).